In [17]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from transformers import AutoTokenizer,AutoModelForMaskedLM
cp = "distilbert-base-multilingual-cased"
model=AutoModelForMaskedLM.from_pretrained(cp)
tokenizer=AutoTokenizer.from_pretrained(cp)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/fb240273126596a03b35c85793d2e82a5b13ac79/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-multilingual-cased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 119547
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-multilingual-cased/snapshots/fb240273126596a03b35c85793d2e82a5b13ac79/pytorch_model.bin
All model checkpoint weights

In [19]:
import datasets
ds=datasets.load_dataset("csv", data_files='/content/drive/Shareddrives/Embible/Data/train_df.csv',sep="\t")

  0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
def tokenize_function(dataset):
  return tokenizer(str(dataset["verse"])
                      )

tokenized_ds=ds.map(tokenize_function)

In [21]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [22]:
from transformers import TrainingArguments


batch_size = 64
# Show the training loss with every epoch
logging_steps = len(ds["train"])

training_args = TrainingArguments(
    output_dir="/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    logging_steps=logging_steps,

)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_ds['train'],
    eval_dataset=tokenized_ds["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [24]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: Unnamed: 0, name, verse_idx, verse. If Unnamed: 0, name, verse_idx, verse are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 22150
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1041
  Number of trainable parameters = 135445755
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using

Epoch,Training Loss,Validation Loss
1,No log,0.714159
2,No log,0.618226
3,No log,0.568545


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: Unnamed: 0, name, verse_idx, verse. If Unnamed: 0, name, verse_idx, verse are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 22150
  Batch size = 64
Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-fi

TrainOutput(global_step=1041, training_loss=0.6944689132294669, metrics={'train_runtime': 1110.5208, 'train_samples_per_second': 59.837, 'train_steps_per_second': 0.937, 'total_flos': 857658793307292.0, 'train_loss': 0.6944689132294669, 'epoch': 3.0})

In [25]:
trainer.save_model("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert")
model.save_pretrained("/content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert")

Saving model checkpoint to /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/pytorch_model.bin
tokenizer config file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/tokenizer_config.json
Special tokens file saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/special_tokens_map.json
Configuration saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/config.json
Model weights saved in /content/drive/Shareddrives/Embible/Bibert_Models/Embibert-finetuned-distilBert/pytorch_model.bin
